In [1]:
from azureml.core import Workspace

In [2]:
subscription_id = "<Enter Azure Subscription #>" #  Azure Subscription
resource_group = "gestaltml" # Name of a logical resource group
workspace_name = "fastai2" # The name of the workspace to look for or to create
workspace_region = 'East US' # Location of the workspace
#experiment_name = 'breastcancer'
score_script = 'score.py'
model_name = 'breastcancerfastai_az01'
service_name = 'breastcancerdetectaz01'

In [3]:
#Create Workspace if needed
# ws = Workspace.create(name=workspace_name,
#                subscription_id=subscription_id,
#                resource_group=resource_group,
#                create_resource_group=True,
#                location=workspace_region,
#                )

In [4]:

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

## Manually Upload & Register Model 
In Azure ML Workspace (or use register script below)

In [5]:
from azureml.core.model import Model

model = Model.register(model_path="export.pkl",
                       model_name=model_name,
                       tags={'area': "breast", 'type': "transfer-learning", 'classes':'Positive,Negative'},
                       description="Breast Cancer Diag-embeded X,Y functions:092220",
                       workspace=ws)

Registering model breastcancerfastai_az01


## Open existing Model
Use if no change to model

In [ ]:
# from azureml.core.model import Model

# #OPen existin model already registered
# model = Model(ws, modelupload_name)

In [6]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.model import InferenceConfig

# myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 'fastai==1.0.61','pillow==5.4.1',
#                                                'torch', 'torchvision>=0.5.0'])


myenv = CondaDependencies.create(pip_packages=[
    'azureml-defaults~=1.13.0', 'fastcore==1.0.0', 'fastprogress==1.0.0', 'fastscript==1.0.0', 'Pillow==5.4.1', 'requests', 'torch==1.6.0', 'torchvision>=0.5.0', 'fastai==2.0.6', 'ipython'
])
 
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script=score_script,
                                   environment=myenv)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults~=1.13.0
  - fastcore==1.0.0
  - fastprogress==1.0.0
  - fastscript==1.0.0
  - Pillow==5.4.1
  - requests
  - torch==1.6.0
  - torchvision>=0.5.0
  - fastai==2.0.6
  - ipython
channels:
- anaconda
- conda-forge



In [7]:
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException



# Remove any existing service under the same name.
try:
   Webservice(ws, service_name).delete()
except WebserviceException:
   pass
    
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=4, 
                                               tags={'data': 'breastcancer',  'method':'transfer learning', 'framework':'pytorch', 'fastai':'2.0.6'},
                                               description='Breast Cancer Diag-embeded X,Y functions:092220')

In [ ]:
#Open existing webservice
#service = Webservice(ws, service_name)


In [8]:

#Deploy new webservice
service = Model.deploy(workspace=ws, 
                           name=service_name, 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

Running..............................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [ ]:
service.get_logs()

In [ ]:
print(service.scoring_uri)

In [ ]:
service.delete()

In [ ]:
service.state